In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

%matplotlib inline

## Reading Baseline Value CSV Files

In [2]:
df_baseline = pd.read_csv("data/Baseline_Values.csv", names=['metric','baseline_value'])
df_baseline

,metric,baseline_value
0,Unique cookies to view course overview page per day:,40000.000000
1,"Unique cookies to click ""Start free trial"" per day:",3200.000000
2,Enrollments per day:,660.000000
3,"Click-through-probability on ""Start free trial"":",0.080000
4,"Probability of enrolling, given click:",0.206250
5,"Probability of payment, given enroll:",0.530000
6,"Probability of payment, given click",0.109313


In [3]:
df_baseline.metric = df_baseline.metric.map(lambda x: x.lower())
df_baseline

,metric,baseline_value
0,unique cookies to view course overview page per day:,40000.000000
1,"unique cookies to click ""start free trial"" per day:",3200.000000
2,enrollments per day:,660.000000
3,"click-through-probability on ""start free trial"":",0.080000
4,"probability of enrolling, given click:",0.206250
5,"probability of payment, given enroll:",0.530000
6,"probability of payment, given click",0.109313


Given the sample size of 5000 cookies visiting the course overview page. 

In [4]:
cookies_sample_size = 5000

## Calculating standard deviation of evaluation metric

**Gross conversion**: That is, number of user-ids to complete checkout and enroll in the free trial divided by number of unique cookies to click the "Start free trial" button. 

In [5]:
total_cookies_per_day = df_baseline.baseline_value[0]
total_cookies_click_start_per_day = df_baseline.baseline_value[1]
P_enroll_given_click = df_baseline.baseline_value[4]

total_click_in_sample = (total_cookies_click_start_per_day/total_cookies_per_day)*cookies_sample_size

Gross_conversion_std = np.sqrt(P_enroll_given_click*(1-P_enroll_given_click)/total_click_in_sample)
Gross_conversion_std = round(Gross_conversion_std,4)
Gross_conversion_std

0.0202

**Retention**: That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by number of user-ids to complete checkout.

In [6]:
total_enrollments_per_day = df_baseline.baseline_value[2]
P_payment_given_enroll = df_baseline.baseline_value[5]

total_enrollments_in_sample = (total_enrollments_per_day/total_cookies_per_day)*cookies_sample_size

Retention_std = np.sqrt(P_payment_given_enroll*(1-P_payment_given_enroll)/total_enrollments_in_sample)
Retention_std = round(Retention_std, 4)
Retention_std

0.0549

**Net conversion**: That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by the number of unique cookies to click the "Start free trial" button.

In [7]:
P_payment_given_click = df_baseline.baseline_value[6]

Net_conversion_std = np.sqrt(P_payment_given_click*(1-P_payment_given_click)/total_click_in_sample)
Net_conversion_std = round(Net_conversion_std, 4)
Net_conversion_std

0.0156

## Sizing

### Number of Samples vs Power

In [8]:
click_to_pageview_rate = total_cookies_click_start_per_day/total_cookies_per_day
click_to_pageview_rate

0.08

In [9]:
enrollment_to_pageview_rate = total_enrollments_per_day/total_cookies_per_day
enrollment_to_pageview_rate

0.0165

## Reading Project Results CSV Files

In [10]:
df_control = pd.read_csv("data/Project_Results_Control.csv")
df_experiment = pd.read_csv("data/Project_Results_Experiment.csv")

In [11]:
df_control.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


In [12]:
df_experiment.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


### Sanity Check

In [13]:
control_summary = pd.Series([df_control.Pageviews.sum(), df_control.Clicks.sum(), df_control.Enrollments.sum(), 
                     df_control.Payments.sum()], index=["pageviews", "clicks", "enrollments", "payments"])
control_summary

pageviews      345543.0
clicks          28378.0
enrollments      3785.0
payments         2033.0
dtype: float64

In [14]:
exp_summary = pd.Series([df_experiment.Pageviews.sum(), df_experiment.Clicks.sum(), df_experiment.Enrollments.sum(), 
                     df_experiment.Payments.sum()], index=["pageviews", "clicks", "enrollments", "payments"])
exp_summary

pageviews      344660.0
clicks          28325.0
enrollments      3423.0
payments         1945.0
dtype: float64

In [15]:
summary = {"control":control_summary, "experiment":exp_summary}
df_results =  pd.DataFrame(summary)
df_results

,control,experiment
pageviews,345543.0,344660.0
clicks,28378.0,28325.0
enrollments,3785.0,3423.0
payments,2033.0,1945.0


For each metric, given the 95% confidence interval for the value that expect to observe. The corresponding z-score is 1.96. 

In [16]:
df_results['total'] = df_results.control + df_results.experiment
df_results['P'] = 0.5
df_results['std'] = np.sqrt(df_results.P*(1-df_results.P)/(df_results.total))
df_results['margin_err'] = df_results['std']*1.96
df_results['lower_bound'] = df_results['P'] - df_results['margin_err']
df_results['higher_bound'] = df_results['P'] + df_results['margin_err']
df_results['obs_con'] = df_results.control/df_results.total
df_results['con_pass_sanity'] = df_results.apply(lambda x: x.obs_con>x.lower_bound and x.obs_con<x.higher_bound, axis=1)
df_results['obs_exp'] = df_results.experiment/df_results.total
df_results['exp_pass_sanity'] = df_results.apply(lambda x: x.obs_exp>x.lower_bound and x.obs_exp<x.higher_bound, axis=1)

df_results 

,control,experiment,total,P,std,margin_err,lower_bound,higher_bound,obs_con,con_pass_sanity,obs_exp,exp_pass_sanity
pageviews,345543.0,344660.0,690203.0,0.5,0.000602,0.001180,0.498820,0.501180,0.500640,True,0.499360,True
clicks,28378.0,28325.0,56703.0,0.5,0.002100,0.004116,0.495884,0.504116,0.500467,True,0.499533,True
enrollments,3785.0,3423.0,7208.0,0.5,0.005889,0.011543,0.488457,0.511543,0.525111,False,0.474889,False
payments,2033.0,1945.0,3978.0,0.5,0.007928,0.015538,0.484462,0.515538,0.511061,True,0.488939,True


**Click-through-probability** (clicks/pageviews) Sanity Check

In [17]:
clicks_con = df_results['control']['clicks']
clicks_exp = df_results['experiment']['clicks']
pageviews_con = df_results['control']['pageviews']
pageviews_exp = df_results['experiment']['pageviews']

#Click-through-probability (CTP)
CTP_con = clicks_con/pageviews_con
CTP_exp = clicks_exp/pageviews_exp

#Standard Error
CTP_SE = np.sqrt(CTP_con*(1-CTP_con)/pageviews_con)

#Margin of error for 95% confidence interval
z = 1.96
CTP_ME = CTP_SE*z

#CI
lower_bound = CTP_con - CTP_ME
higher_bound = CTP_con + CTP_ME

print(CTP_con, CTP_exp, lower_bound, higher_bound)

print(CTP_exp>lower_bound and CTP_exp<higher_bound)

0.08212581357457682 0.08218244066616376 0.08121035975252971 0.08304126739662393
True


## Characteristics of the datasets

In [18]:
df_control.shape

(37, 5)

**Missing Data**

In [19]:
df_control.isnull().sum()

Date            0
Pageviews       0
Clicks          0
Enrollments    14
Payments       14
dtype: int64

In [20]:
df_experiment.isnull().sum()

Date            0
Pageviews       0
Clicks          0
Enrollments    14
Payments       14
dtype: int64

**Drop Nulls**

In [21]:
df_control.dropna(inplace=True)
df_control.isnull().any()

Date           False
Pageviews      False
Clicks         False
Enrollments    False
Payments       False
dtype: bool

In [22]:
df_experiment.dropna(inplace=True)
df_experiment.isnull().any()

Date           False
Pageviews      False
Clicks         False
Enrollments    False
Payments       False
dtype: bool

**Edit Data Frame**

In [25]:
results_NoNull = {"control": pd.Series([df_control.Pageviews.sum(), df_control.Clicks.sum(), df_control.Enrollments.sum(), 
                     df_control.Payments.sum()], index=['pageviews','clicks','enrollments','payments']),
                  "experiment": pd.Series([df_experiment.Pageviews.sum(), df_experiment.Clicks.sum(), df_experiment.Enrollments.sum(), 
                     df_experiment.Payments.sum()], index=['pageviews','clicks','enrollments','payments'])}

In [26]:
df_results_e = pd.DataFrame(results_NoNull)
df_results_e

,control,experiment
pageviews,212163.0,211362.0
clicks,17293.0,17260.0
enrollments,3785.0,3423.0
payments,2033.0,1945.0


In [27]:
df_results_e['total'] = df_results_e['control'] + df_results_e['experiment']
df_results_e

,control,experiment,total
pageviews,212163.0,211362.0,423525.0
clicks,17293.0,17260.0,34553.0
enrollments,3785.0,3423.0,7208.0
payments,2033.0,1945.0,3978.0


### Evaluation Metrics Analysis

In [30]:
# control group
clicks_con = df_results_e['control']['clicks']
enrollments_con = df_results_e['control']['enrollments']
payments_con = df_results_e['control']['payments']

grossConversion_con = enrollments_con/clicks_con
netConversion_con = payments_con/clicks_con

# experiment group
clicks_exp = df_results_e['experiment']['clicks']
enrollments_exp = df_results_e['experiment']['enrollments']
payments_exp = df_results_e['experiment']['payments']

grossConversion_exp = enrollments_exp/clicks_exp
netConversion_exp = payments_exp/clicks_exp

# Mean
clicks_tot = df_results_e['total']['clicks']
enrollments_tot = df_results_e['total']['enrollments']
payments_tot = df_results_e['total']['payments']

grossConversion = enrollments_tot/clicks_tot
netConversion = payments_tot/clicks_tot

In [34]:
pd.DataFrame([[grossConversion_con,grossConversion_exp,grossConversion], [netConversion_con,netConversion_exp,netConversion]],
            columns = ['control','experiment','mean'],index=['Gross Conversion','Net Conversion'])

,control,experiment,mean
Gross Conversion,0.218875,0.198320,0.208607
Net Conversion,0.117562,0.112688,0.115127


In [37]:
def stat_cal(p_hat, N_con, N_exp, z_score, obs_diff):
    std_err = np.sqrt(p_hat*(1-p_hat)*(1/N_con+1/N_exp))
    margin_err = std_err*z_score
    lower_bound = obs_diff - margin_err
    higher_bound = obs_diff + margin_err
    return std_err, margin_err, lower_bound, higher_bound

In [42]:
grossConversion_diff = grossConversion_exp - grossConversion_con
netConversion_diff = netConversion_exp - netConversion_con
grossConversion_diff, netConversion_diff

(-0.020554874580361565, -0.0048737226745441675)

In [40]:
gross_se, gross_me, gross_CI_l, gross_CI_h = stat_cal(grossConversion, clicks_con, clicks_exp, 1.96, grossConversion_diff)

net_se, net_me, net_CI_l, net_CI_h = stat_cal(netConversion, clicks_con, clicks_exp, 1.96, netConversion_diff)

In [41]:
pd.DataFrame([[gross_se, gross_me, gross_CI_l, gross_CI_h],[net_se, net_me, net_CI_l, net_CI_h]], 
             columns = ['SE','ME','CI_lower','CI_higher'], index = ['Gross Conversion','Net Conversion'])

,SE,ME,CI_lower,CI_higher
Gross Conversion,0.004372,0.008568,-0.029123,-0.011986
Net Conversion,0.003434,0.006731,-0.011605,0.001857
